In [ ]:
import os
import leafmap
import numpy as np
import rasterio

In [ ]:
dem = r"INPUT FILE PATH"
# Import DEM


In [ ]:
wbt = leafmap.WhiteboxTools()

In [ ]:
wbt.set_working_dir(os.getcwd())
wbt.verbose = False

In [ ]:
os.environ["NODATA"] = "-32768"

In [ ]:
m = leafmap.Map()
m

In [ ]:
m.add_raster(dem, palette="terrain", layer_name="DEM")
m

In [ ]:
leafmap.image_min_max(dem)

In [ ]:
m.add_colormap(cmap="terrain", vmin=-8, vmax=680, label="Elevation (m)")

In [ ]:
wbt.feature_preserving_smoothing(dem, "smoothed.tif", filter=9)

In [ ]:
Kogi_bound = r"INPUT FILE PATH"
# Add AOI Boundry

In [ ]:
m = leafmap.Map()
m.add_basemap("Satellite")
m.add_raster("smoothed.tif", colormap="terrain", nodata=32767, layer_name="Smoothed DEM")
m.add_shp(Kogi_bound, layer_name="AOI", info_mode=None)
m

In [ ]:
wbt.hillshade("smoothed.tif", "hillshade.tif", azimuth=315, altitude=35)
# create Hillshade
m.add_raster("hillshade.tif", layer_name="Hillshade")
m.layers[-1].opacity = 0.6
# Visualize Hill shade

In [ ]:
wbt.fill_depressions("smoothed.tif", "filled.tif")

In [ ]:
wbt.breach_depressions("smoothed.tif", "breached.tif")

In [ ]:
wbt.find_no_flow_cells("breached.tif", "noflow2.tif")

In [ ]:
wbt.d8_pointer("breached.tif", "flow_direction.tif")

In [ ]:
wbt.d8_flow_accumulation("breached.tif", "flow_accum.tif")

In [ ]:
m.add_raster("flow_accum.tif", layer_name="Flow Accumulation")

In [ ]:
wbt.extract_streams("flow_accum.tif", "streams.tif", threshold=5000)

In [ ]:
m.layers[-1].visible = False
m.add_raster("streams.tif", layer_name="Streams")

In [ ]:
wbt.raster_streams_to_vector(
    "streams.tif", d8_pntr="flow_direction.tif", output="streams.shp"
)

In [ ]:
m.add_raster("flow_direction.tif", layer_name="Flow Direction", visible = False)

In [ ]:
wbt.raster_streams_to_vector(
    "streams.tif", d8_pntr="flow_direction.tif", output="streams.shp"
)

In [ ]:
leafmap.vector_set_crs(source="streams.shp", output="streams.shp", crs="EPSG:32632")

In [ ]:
m.add_shp(
    "streams.shp",
    layer_name="Streams Vector",
    style={"color": "#ff0000", "weight": 3},
    info_mode=None,
)
m